In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime, time
import skimage.io
import skimage.transform
import json

import modutils

In [89]:
def food101_readmeta(fname):
    with open(fname) as f:
        f_meta = json.loads(f.read())
    f_meta = zip(*[(x,z) for (x, y) in f_meta.items() for z in y])
    (rlab, rfn) = tuple(np.array(x) for x in f_meta)
    fullmap = list(enumerate(set(rlab)))
    labmap = {y:x for (x,y) in fullmap}
    ry = np.array([labmap[x] for x in rlab])
    return (rfn, ry, fullmap)

def food101_readimg(fname, req_size = 512, out_size = 512):
    f_img = skimage.io.imread(fname)
    if len(f_img.shape) != 3:
        print('Invalid dimensions in file "{0}"'.format(fname))
        return None
    misdim = req_size - f_img.shape[0]
    if misdim < 0:
        raise 'error'
    if misdim > 0:
        addtop = misdim // 2
        addbot = misdim - addtop
        if addtop > 0:
            f_img = np.vstack([np.tile(f_img[0], addtop).reshape(f_img.shape[1], addtop, 3).transpose([1,0,2]),
                                f_img,
                                np.tile(f_img[-1], addbot).reshape(f_img.shape[1], addbot, 3).transpose([1,0,2])])
        else:
            f_img = np.vstack([f_img, np.tile(f_img[-1], addbot).reshape(f_img.shape[1], addbot, 3).transpose([1,0,2])])
            
    misdim = req_size - f_img.shape[1]
    if misdim < 0:
        raise 'error'
    fin_res = f_img
    if misdim > 0:
        addl = misdim // 2
        addr = misdim - addl
        if addl > 0:
            fin_res = np.hstack([np.tile(f_img[:, 0, :], addl).reshape(f_img.shape[0], addl, 3),
                              f_img,
                              np.tile(f_img[:, -1, :], addr).reshape(f_img.shape[0], addr, 3)])
        else:
            fin_res = np.hstack([f_img, np.tile(f_img[:, -1, :], addr).reshape(f_img.shape[0], addr, 3)])
    if out_size == req_size:
        return fin_res / 255.0
    return skimage.transform.resize(fin_res, output_shape=(out_size, out_size), mode='constant', order = 1) / 255.0

In [158]:
%%time
dev_met, dev_y, _ = food101_readmeta('../DataSets/food-101/meta/train.json')
imgpath = '../DataSets/food-101/images/{0}.jpg'
imgpath128 = '../DataSets/food-101/images/{0}s128.jpg'
imgpath032 = '../DataSets/food-101/images/{0}s032.jpg'
ids = np.array(range(len(dev_y)))
np.random.shuffle(ids)

(train_met, train_y),(valid_met, valid_y) = modutils.splitSample((dev_met[ids], dev_y[ids]),[0.8, 0.2])

Wall time: 199 ms


In [64]:
#Resave train-valid files with 128x128 resolution (approx. 45 minutes)
for i in range(15802, len(dev_met)):
    fimg = food101_readimg(imgpath.format(dev_met[i]), out_size=128)
    if fimg is None:
        continue
    skimage.io.imsave(imgpath128.format(dev_met[i]), fimg)
    print('Processed {0} out of {1} files...'.format(i, len(dev_met)), end='\r')

Invalid dimensions in file "../DataSets/food-101/images/steak/1340977.jpg"


C:\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Invalid dimensions in file "../DataSets/food-101/images/bread_pudding/1375816.jpg"
Invalid dimensions in file "../DataSets/food-101/images/lasagna/3787908.jpg"


C:\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ../DataSets/food-101/images/breakfast_burrito/462294s128.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


In [161]:
#Resave train-valid files with 128x128 resolution (approx. 45 minutes)
for i in range(len(dev_met)):
    fimg = food101_readimg(imgpath128.format(dev_met[i]), req_size=128, out_size=32)
    if fimg is None:
        continue
    skimage.io.imsave(imgpath032.format(dev_met[i]), fimg * 255.0)
    print('Processed {0} out of {1} files...'.format(i, len(dev_met)), end='\r')

C:\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


C:\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ../DataSets/food-101/images/bread_pudding/444890s032.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


In [44]:
%%timeit
fimg = food101_readimg(imgpath.format(train_met[10]), out_size=128)

100 loops, best of 3: 16.6 ms per loop


In [48]:
%%timeit
fimg = food101_readimg('test128.jpg', req_size=128, out_size=128)

1000 loops, best of 3: 485 µs per loop


In [46]:
skimage.io.imsave('test128.jpg', fimg)

C:\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [162]:
#simple logistic regression
CONST_SZ = 32
CONST_LBL = 101

tf.reset_default_graph()

with tf.name_scope('Input'):
    tf_x = tf.placeholder(shape=(None, CONST_SZ, CONST_SZ, 3), dtype=tf.float32, name='x')
    tf_y = tf.placeholder(shape=(None,), dtype=tf.int32, name='y')
    tf_xflat = tf.reshape(tf_x, shape=(tf.shape(tf_x)[0], CONST_SZ * CONST_SZ * 3))
    tf_y1hot = tf.one_hot(tf_y, CONST_LBL, name='y1hot')
    
tf_lgt = tf.layers.dense(tf_xflat, CONST_LBL)
tf_prob = tf.nn.softmax(tf_lgt, name='prob')
tf_pred = tf.reshape(tf.nn.top_k(tf_prob, name='pred')[1], (tf.shape(tf_prob)[0],))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf_pred, tf_y), tf.float32))

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y1hot, logits=tf_lgt))
tf_train = tf.train.AdamOptimizer(1e-3).minimize(tf_loss)

tfsaver = tf.train.Saver(max_to_keep=2)

dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/18Food01_32LOGREG-{0}'.format(dt_now), tf.get_default_graph())
print('Graph created')

Graph created


In [164]:
#simple cnn
CONST_SZ = 32
CONST_LBL = 101

tf.reset_default_graph()

with tf.name_scope('Input'):
    tf_x = tf.placeholder(shape=(None, CONST_SZ, CONST_SZ, 3), dtype=tf.float32, name='x')
    tf_y = tf.placeholder(shape=(None,), dtype=tf.int32, name='y')
    tf_y1hot = tf.one_hot(tf_y, CONST_LBL, name='y1hot')
    
tf_cnn0 = tf.layers.conv2d(tf_x, 30, 3, padding='same', activation=tf.nn.elu)
tf_mp0 = tf.layers.max_pooling2d(tf_cnn0, 2, 2, padding='same')
#16x16x30
tf_cnn1 = tf.layers.conv2d(tf_mp0, 50, 3, padding='same', activation=tf.nn.elu)
tf_mp1 = tf.layers.max_pooling2d(tf_cnn1, 2, 2, padding='same')
#8x8x50
tf_cnn2 = tf.layers.conv2d(tf_mp1, 100, 3, padding='same', activation=tf.nn.elu)
tf_mp2 = tf.layers.max_pooling2d(tf_cnn2, 2, 2, padding='same')
#4x4x100
tf_cnnout = tf.reshape(tf_mp2, shape=(tf.shape(tf_x)[0], 4*4*100))

tf_lgt0 = tf.layers.dense(tf_cnnout, 200, activation=tf.nn.elu)
tf_lgt = tf.layers.dense(tf_lgt0, CONST_LBL)

tf_prob = tf.nn.softmax(tf_lgt, name='prob')
tf_pred = tf.reshape(tf.nn.top_k(tf_prob, name='pred')[1], (tf.shape(tf_prob)[0],))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf_pred, tf_y), tf.float32))

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y1hot, logits=tf_lgt))
tf_train = tf.train.AdamOptimizer(1e-3).minimize(tf_loss)

tfsaver = tf.train.Saver(max_to_keep=2)

dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/18Food01_01LOGREG-{0}'.format(dt_now), tf.get_default_graph())
print('Graph created')

Graph created


In [165]:
batch_size = 200
num_steps  = 1
num_epochs = 10
used_imgpath = imgpath032

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f}'
fmtstr_run = 'Minibatch ({2}/{3}, {4:.2f} loss): {0:.1f} sec (IO), {1:.1f} sec (TRAIN)'
train_tuple = (train_met, train_y)
valid_tuple = (valid_met, valid_y)
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        te0 = time.perf_counter()
        ind_minibatch = 0
        num_minibatches = len(train_y) // batch_size
        if num_minibatches * batch_size < len(train_y):
            num_minibatches += 1
        for (mini_met, mini_y) in modutils.shuffleBatches(train_tuple, batchSize=batch_size):
            tmb0 = time.perf_counter()
            
            mini_x = np.array([food101_readimg(used_imgpath.format(x), req_size=CONST_SZ, out_size=CONST_SZ) for x in mini_met])
            train_batch = {tf_x:mini_x, tf_y:mini_y}
        
            tmb1 = time.perf_counter()
            for j in range(num_steps):
                tf_train.run(feed_dict=train_batch)
            tmb2 = time.perf_counter()
            trloss = tf_loss.eval(train_batch)
            print(fmtstr_run.format(tmb1-tmb0, tmb2-tmb1, ind_minibatch, num_minibatches, trloss), end='\r')
            ind_minibatch += 1
    
        te1 = time.perf_counter()
        vloss = 0
        vacc  = 0
        ind_minibatch = 0
        num_minibatches = len(valid_y) // batch_size
        if num_minibatches * batch_size < len(valid_y):
            num_minibatches += 1
        for (mv_met, mv_y) in modutils.shuffleBatches(valid_tuple, batchSize=batch_size):
            mv_x = np.array([food101_readimg(used_imgpath.format(x), req_size=CONST_SZ, out_size=CONST_SZ) for x in mv_met])
            valid_batch = {tf_x:mv_x, tf_y:mv_y}
            
            tloss, tacc = tfs.run([tf_loss, tf_accuracy], valid_batch)
            vloss += tloss * batch_size
            vacc += vacc * batch_size
            
            print(fmtstr_run.format(tmb1-tmb0, tmb2-tmb1, ind_minibatch, num_minibatches, tloss), end='\r')
            ind_minibatch += 1
            #tffw.add_summary(summary, i)
            #if i%checkpoints == 0 and i > 0:
        lv = vloss / len(valid_y)
        ac = vacc / len(valid_y)
        p = tfsaver.save(tfs, 'D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-{0:04d}.ckpt'.format(i))
        print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac))

Model saved at checkpoint: D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-0000.ckpt
Epoch 0 (1.08e+03 sec): 		VL:4.006		AC:0.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-0001.ckpt
Epoch 1 (2.45e+02 sec): 		VL:3.800		AC:0.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-0002.ckpt
Epoch 2 (2.41e+02 sec): 		VL:3.747		AC:0.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-0003.ckpt
Epoch 3 (2.4e+02 sec): 		VL:3.640		AC:0.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-0004.ckpt
Epoch 4 (2.62e+02 sec): 		VL:3.651		AC:0.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-18FOOD01/model01CNN32-0005.ckpt
Epoch 5 (2.54e+02 sec): 		VL:3.673		AC:0.000


KeyboardInterrupt: 

In [157]:
batch_size = 200
num_steps  = 1
num_epochs = 10
used_imgpath = imgpath128

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f}'
fmtstr_run = 'Minibatch ({2}/{3}, {4:.2f} loss): {0:.1f} sec (IO), {1:.1f} sec (TRAIN)'
train_tuple = (train_met, train_y)
valid_tuple = (valid_met, valid_y)
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    tfsaver.restore(tfs, 'D:/Jupyter/mltest/Models-18FOOD01/model01CNN1-0000.ckpt')
    for i in range(1, num_epochs):
        te0 = time.perf_counter()
        ind_minibatch = 0
        num_minibatches = len(train_y) // batch_size
        if num_minibatches * batch_size < len(train_y):
            num_minibatches += 1
        for (mini_met, mini_y) in modutils.shuffleBatches(train_tuple, batchSize=batch_size):
            tmb0 = time.perf_counter()
            
            mini_x = np.array([food101_readimg(used_imgpath.format(x), req_size=CONST_SZ, out_size=CONST_SZ) for x in mini_met])
            train_batch = {tf_x:mini_x, tf_y:mini_y}
        
            tmb1 = time.perf_counter()
            for j in range(num_steps):
                tf_train.run(feed_dict=train_batch)
            tmb2 = time.perf_counter()
            trloss = tf_loss.eval(train_batch)
            print(fmtstr_run.format(tmb1-tmb0, tmb2-tmb1, ind_minibatch, num_minibatches, trloss), end='\r')
            ind_minibatch += 1
    
        te1 = time.perf_counter()
        vloss = 0
        vacc  = 0
        ind_minibatch = 0
        num_minibatches = len(valid_y) // batch_size
        if num_minibatches * batch_size < len(valid_y):
            num_minibatches += 1
        for (mv_met, mv_y) in modutils.shuffleBatches(valid_tuple, batchSize=batch_size):
            mv_x = np.array([food101_readimg(used_imgpath.format(x), req_size=CONST_SZ, out_size=CONST_SZ) for x in mv_met])
            valid_batch = {tf_x:mv_x, tf_y:mv_y}
            
            tloss, tacc = tfs.run([tf_loss, tf_accuracy], valid_batch)
            vloss += tloss * batch_size
            vacc += vacc * batch_size
            
            print(fmtstr_run.format(tmb1-tmb0, tmb2-tmb1, ind_minibatch, num_minibatches, tloss), end='\r')
            ind_minibatch += 1
            #tffw.add_summary(summary, i)
            #if i%checkpoints == 0 and i > 0:
        lv = vloss / len(valid_y)
        ac = vacc / len(valid_y)
        p = tfsaver.save(tfs, 'D:/Jupyter/mltest/Models-18FOOD01/model01CNN1-{0:04d}.ckpt'.format(i))
        print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac))

INFO:tensorflow:Restoring parameters from D:/Jupyter/mltest/Models-18FOOD01/model01CNN1-0000.ckpt


KeyboardInterrupt: 

In [ ]:
batch_size = 200
num_steps  = 1
num_epochs = 10
used_imgpath = imgpath128

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f}'
fmtstr_run = 'Minibatch ({2}/{3}, {4:.2f} loss): {0:.1f} sec (IO), {1:.1f} sec (TRAIN)'
train_tuple = (train_met, train_y)
valid_tuple = (valid_met, valid_y)
valid_p = None
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    tfsaver.restore(tfs, 'D:/Jupyter/mltest/Models-18FOOD01/model01CNN1-0000.ckpt')
    ind_minibatch = 0
    num_minibatches = len(valid_y) // batch_size
    if num_minibatches * batch_size < len(valid_y):
        num_minibatches += 1
    for (mv_met, mv_y) in modutils.shuffleBatches(valid_tuple, batchSize=batch_size):
        t0 = time.perf_counter()
        mv_x = np.array([food101_readimg(used_imgpath.format(x), req_size=CONST_SZ, out_size=CONST_SZ) for x in mv_met])
        valid_batch = {tf_x:mv_x, tf_y:mv_y}
        t1 = time.perf_counter()
        valid_p0 = tf_prob.eval(valid_batch)
        t2 = time.perf_counter()
        if valid_p is None:
            valid_p = valid_p0
        else:
            valid_p = np.vstack([valid_p, valid_p0])
        t3 = time.perf_counter()
        ind_minibatch += 1
        print('{0} of {1} done; IO: {2:.1f} sec, TF: {3:.1f} sec, NP: {4:.1f} sec'.format(ind_minibatch, num_minibatches, t1-t0,t2-t1,t3-t2), end='\r')

In [132]:
valid_y[12], sorted(list(enumerate(valid_p[12])), key=lambda x: x[1], reverse=True)[:10]

(81,
 [(23, 0.10276058),
  (49, 0.093272746),
  (88, 0.065886937),
  (67, 0.06420172),
  (71, 0.052962728),
  (27, 0.043844711),
  (90, 0.034535103),
  (54, 0.029794814),
  (8, 0.028898258),
  (89, 0.028106257)])

In [135]:
valid_top = [np.sum(valid_p[i] >= valid_p[i][valid_y[i]]) for i in range(len(valid_y))]

In [140]:
np.mean(np.array(valid_top))

50.783270978086584

In [141]:
valid_top[:10]

[5, 88, 43, 30, 5, 10, 79, 56, 39, 65]

In [155]:
np.mean(np.array(valid_top)[(np.min(valid_p, axis=1) < 0.0005)])

50.7826310758144